In [1]:
import csv
import os
import librosa
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
def feature_extract_spectogram(y, sr, voice_id):
    features = []

    features.append(voice_id)  # 1

    # zero crossing
    zero_crossing = librosa.zero_crossings(y, pad=False)
    features.append(zero_crossing.sum())  # f0

    # spectral centroids
    spectral_centroids = librosa.feature.spectral_centroid(y, sr=sr)[0]
    features.append(spectral_centroids.mean())  # f1
    #features.append(spectral_centroids.var())  # f2

    # spectral rolloff
    spectral_rolloff = librosa.feature.spectral_rolloff(y, sr=sr)[0]
    features.append(spectral_rolloff.mean())  # f3
    #features.append(spectral_rolloff.var())  # f4

    # Chroma Frequencies
    chromagram = librosa.feature.chroma_stft(y, sr=sr)
    features += chromagram.mean(axis=1).tolist()  # f5 - f16
    #features += chromagram.var(axis=1).tolist()  # f17 - f28

    # Mel-Frequency Cepstral Coefficients
    MFCCs = librosa.feature.mfcc(y, sr=sr,n_mfcc=50)
    features += MFCCs.mean(axis=1).tolist()  # f29 - f48
    #features += MFCCs.var(axis=1).tolist()  # f49 - f68

    return features

In [3]:
def feature_extract_rythm(y,sr,voice_id):
    features = []
    features.append(voice_id)


    tempogram = abs(librosa.feature.fourier_tempogram(y, sr=sr))
    features += tempogram.mean(axis=1).tolist()   #193

    
    return features

In [4]:
def header_name(name,num):
    names_list=[]
    for i in range(num):
        temp=name+str(i)
        names_list.append(temp)
    return names_list


In [5]:
rows_spectrum=[]
rows_rythm=[]

voices_path = "D:\\Term6\\Machine Learning\\final_last_term\\Copy(1)"
csv_path = "C:\\Users\\admin\\OneDrive\\Desktop\\KARAJ_ML\\dataV4.csv"

file_names = os.listdir(voices_path)

df = pd.read_csv(csv_path)
df = df.values
shapee = (df.shape)[1]
voice_ids = df[:,(shapee-1)]
df = np.delete(df, (shapee-1), 1)

In [6]:
feature1_list=["voice_id","zero_crossing","spect_cent_mean","spect_rol_mean",]
chrom_mean_list=header_name("chrom_mean_",12)
#chrom_var_list=header_name("chrom_var_",12)
mfcc_mean_list=header_name("mfcc_mean_",50)
#mfcc_var_list=header_name("mfcc_var_",20)
spectrogram_feature_list=feature1_list+chrom_mean_list+mfcc_mean_list + ["emotion_ID","Text_ID","Sex","Age"]

In [7]:
voiceid_list=["voice_id"]
rythm_mean_list=header_name("tempo_mean_",193)

rythmic_feature_list=voiceid_list+rythm_mean_list

In [9]:
for i in range(len(voice_ids)):
    name = str(voice_ids[i])
    voice_path_temp = voices_path + '\\' + name + '.wav'
    if os.path.exists(voice_path_temp):
        temp_data = list(df[i])
        y, sr=librosa.load(voice_path_temp)
        feature_temp_spectrum=feature_extract_spectogram(y,sr,name)
        feature_temp_spectrum += temp_data
        rows_spectrum.append(feature_temp_spectrum)
        feature_temp_rythm=feature_extract_rythm(y,sr,name)
        rows_rythm.append(feature_temp_rythm)

In [10]:
df_spectrum = pd.DataFrame(rows_spectrum, columns=spectrogram_feature_list)

df_spectrum.to_csv('C:\\Users\\admin\\OneDrive\\Desktop\\KARAJ_ML\\features_spectrum_final.csv', index = False)

df_rythm = pd.DataFrame(rows_rythm, columns=rythmic_feature_list)

df_rythm.to_csv('C:\\Users\\admin\\OneDrive\\Desktop\\KARAJ_ML\\features_rythmic_final.csv', index = False)

